In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from category_encoders import TargetEncoder
from xgboost import XGBRegressor

train = pd.read_csv('data/challenge3_train.csv', index_col='id')
test = pd.read_csv('data/challenge3_test.csv', index_col='id')
full_data = pd.concat([train, test]).drop(columns=['target'])

features_bin = ['f0', 'f26'] # binary
features_nom_low = ['f5'] # nominal low cardinality (<=3)
features_nom_high = ['f12', 'f28'] # >= 26
features_ord_low = ['f1', 'f2', 'f3', 'f6', 'f9', 'f11', 'f13', 'f15', 'f17', 'f18', 'f19', 'f20', 'f21', 'f23', 'f24', 'f25', 'f27', 'f29']
features_ord_num = ['f1', 'f2', 'f3', 'f6', 'f9', 'f11', 'f13', 'f17', 'f18', 'f19', 'f21', 'f23', 'f24', 'f25', 'f27', 'f29']
features_ord_high = ['f4', 'f10', 'f14']
features_real = ['f7', 'f8', 'f16', 'f22', 'f30'] # real numbers.


features_ord_alph = ['f15', 'f20']
ohe_columns = features_ord_alph + features_nom_low


# Cleanup. This is doing data leakage though, when using information from test set..
# Fix 0 value noise, change to most common value in column. 
impute_0_columns = ['f3', 'f18', 'f21']

for column in impute_0_columns:
    train.loc[train[column] == 0, column] = train[column].mode() # mode() finds most common value
    test.loc[test[column] == 0, column] = test[column].mode()

# remove -1 from 'month' column f11
train.loc[train['f11'] == -1, 'f11'] = train['f11'].mode()
test.loc[test['f11'] == -1, 'f11'] = test['f11'].mode()


# Encoding
# Alphabetic -> numbers (a=1, b=2 etc). f15, f20
for col in features_ord_alph:
    ord_order_dict = {i : j for j, i in enumerate(sorted(list(set(list(train[col].dropna().unique()) + list(test[col].dropna().unique())))))}
    train[f'{col}_en'] = train[col].fillna('NULL').map(ord_order_dict)
    test[f'{col}_en'] = test[col].fillna('NULL').map(ord_order_dict)

    
# One hot encoding
train = pd.get_dummies(train, columns=ohe_columns + features_nom_high)
test = pd.get_dummies(test, columns=ohe_columns)


# add cyclical feature (f11 - month)
def cyc_enc(df, col, max_vals):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_vals)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_vals)
    return df
train = cyc_enc(train, 'f11', 11)
test = cyc_enc(test, 'f11', 11)

# split train set into a train and test set
X = train.drop(['target'], axis=1)
y = train['target']

train, test, target, y_test = train_test_split(X, y)


In [61]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as scaler
from sklearn.linear_model import LogisticRegression as lr
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix 
import seaborn as sns
import matplotlib.pyplot as plt


scaler = StandardScaler()
lr = LogisticRegression()
model1 = Pipeline([('standardize', scaler),
                    ('log_reg', lr)])



model1.fit(X_train, y_train)

predictions = model1.predict(X_test)
score = roc_auc_score(predictions, y_test)
print(score)

0.498851649706631


AttributeError: 'numpy.ndarray' object has no attribute 'head'